# Some Python Identities

## len(x) ≡ x.\_\_len\_\_()

In [1]:
class Fixed_size:
    def __len__(self):
        return 42

In [2]:
fs = Fixed_size()
len(fs), fs.__len__()

(42, 42)

## x.method(\*args, \*\*kw) ≡ (x.\_\_class\_\_).method(x, \*args, \*\*kw)

In [3]:
class MyClass1:
    def method(self, positional, keyword="keyword"):
        print(self, positional, keyword)

In [4]:
mc = MyClass1()
mc.method("hello", "world")

<__main__.MyClass1 object at 0x7f6bf002e588> hello world


In [5]:
mc.__class__

__main__.MyClass1

In [6]:
mc.__class__.method

<function __main__.MyClass1.method>

In [7]:
mc.__class__.method(mc, "hello", "world")

<__main__.MyClass1 object at 0x7f6bf002e588> hello world


In [8]:
fs.__len__(), fs.__class__.__len__(fs), fs.__class__ is Fixed_size

(42, 42, True)

## x in y ≡ y.\_\_contains\_\_(x)

In [9]:
("BCD" in "ABCDEF", "ABCDEF".__contains__("BCD"))

(True, True)

In [10]:
class SearchableListOfLists:
    def __init__(self, *args):
        self._values = args
    def __contains__(self, value):
        return any(value in thing for thing in self._values)

In [11]:
sl = SearchableListOfLists([1, 2, 3], [4, 5, 6], [7, 8, 9])
[i in sl for i in range(11)]

[False, True, True, True, True, True, True, True, True, True, False]

## items[key] ≡ items.\_\_getitem\_\_(key)

In [12]:
class MyContainer:
    def __init__(self, value):
        self.value = value
    def __getitem__(self, key):
        print("Key:", key)
        return 42

In [13]:
mc = MyContainer(", world")
mc["hello"], mc["goodbye"]

Key: hello
Key: goodbye


(42, 42)

In [14]:
mc[1:]

Key: slice(1, None, None)


42

In [15]:
"abcdefghijklmnopqrstuvwxyz"[slice(1, None, None)]

'bcdefghijklmnopqrstuvwxyz'

In [16]:
mc[20:1:]

Key: slice(20, 1, None)


42

In [17]:
mc[20:1:-2]

Key: slice(20, 1, -2)


42

In [18]:
"abcdefghijklmnopqrstuvwxyz"[slice(20, 1, -2)]

'usqomkigec'

## items[key] = value≡ items.\_\_setitem\_\_(key, value)

In [19]:
class SettableItems:
    def __init__(self, v):
        self._list = []
        self.v = v
    def __setitem__(self, key, value):
        self._list.append((self.v, key, value))
    def dump(self):
        for item in self._list:
            print(item)

In [20]:
s = SettableItems("@@")
s["hello"] = "World"
s["farewell"] = "Cruel world"
s.dump()

('@@', 'hello', 'World')
('@@', 'farewell', 'Cruel world')


## f(\*args, \*\*kwargs) ≡ f.\_\_call\_\_(\*args, \*\*kwargs)

In [21]:
s1 = SettableItems.__call__("££")
s1[1] = 2
s1[3] = 4
s1.dump()

('££', 1, 2)
('££', 3, 4)


## Therefore ...
## f(\*args, \*\*kwargs) ≡ f\_\_class\_\_.\_\_call\_\_(f, \*args, \*\*kwargs)

In [22]:
def f(x, y):
    return x+y

f("Hello", "world")

'Helloworld'

In [23]:
f.__call__("Hello", "world")

'Helloworld'

In [24]:
f.__call__.__call__("Hello", "world")

'Helloworld'

In [25]:
f.__call__.__call__.__call__("Hello", "world")

'Helloworld'

In [26]:
f.__call__.__call__.__call__.__call__("Hello", "world")

'Helloworld'

In [27]:
f.__class__, type(f)

(function, function)

In [28]:
f.__class__.__call__(f, "Hello", "world")

'Helloworld'

In [29]:
SettableItems.__class__

type

In [30]:
s2 = type.__call__(SettableItems, "##")
s2[1] = "one"
s2[2] = "two"
s2.dump()

('##', 1, 'one')
('##', 2, 'two')


# And finally ...

## class X(base1, base2, ...): ... ≡ X = type('X', (base1, base2, ...), { definitions_dict})

In [31]:
def m(self, other):
    return self is other

In [32]:
T = type('T++', (object, ), {"is_same_as": m})
T

__main__.T++

In [33]:
t = T()
t

<__main__.T++ at 0x7f6bf0040be0>

In [34]:
t.is_same_as(None), t.is_same_as(t)

(False, True)

In [35]:
class print_args(type):
    def __new__(cls, name, bases, name_space):
        print("class:", cls)
        print("Declaring class", name)
        print("Bases:", bases)
        print("Namespace:", name_space)
        return type.__new__(cls, name, bases, name_space)

In [36]:
class A: pass

class B: pass

In [37]:
print("Defining C")
class C(A, B, metaclass=print_args):
    CVar = "class attribute"
    def myMethod(self, a):
        return self.N, self.CVar
    def __init__(self, N):
        self.N = N

Defining C
class: <class '__main__.print_args'>
Declaring class C
Bases: (<class '__main__.A'>, <class '__main__.B'>)
Namespace: {'__init__': <function C.__init__ at 0x7f6bf004d620>, '__qualname__': 'C', 'CVar': 'class attribute', 'myMethod': <function C.myMethod at 0x7f6bf00268c8>, '__module__': '__main__'}


In [38]:
print("Creating c")
c = C(42)
print("CVars:", C.CVar, c.CVar)
print("C:", C.__dict__)
print("c:", c.__dict__)

Creating c
CVars: class attribute class attribute
C: {'CVar': 'class attribute', '__module__': '__main__', '__doc__': None, '__init__': <function C.__init__ at 0x7f6bf004d620>, 'myMethod': <function C.myMethod at 0x7f6bf00268c8>}
c: {'N': 42}


In [39]:
c.CVar = "instance attribute"
print("CVars:", c.CVar, C.CVar)
print("c:", c.__dict__)

CVars: instance attribute class attribute
c: {'N': 42, 'CVar': 'instance attribute'}
